In [1]:
import sys
sys.path.append('../../pyutils')

import numpy as np
from scipy.linalg import solve_triangular

import metrics

# The Cholesky decomposition

Let $A \in \mathbb{R}^{n*n}$ a symmetric positive definite matrix.  
$A$ can de decomposed as:
$$A = LL^T$$
with $L \in \mathbb{R}^{n*n}$ a lower triangular matrix with positive diagonal entries.


$$L_{jj} = \sqrt{A_{jj} - \sum_{k=1}^{j-1} L_{jk}^2}$$
$$L_{ij} = \frac{1}{L_{jj}}(A_{ij} - \sum_{k=1}^{j-1}L_{ik}L_{jk}) \text{ for } i > j$$

The algorithm can be impleted row by row or column by column

In [2]:
def cholesky(A):
    n = len(A)
    L = np.zeros((n, n))
    
    for j in range(n):
        L[j,j] = np.sqrt(A[j,j] - np.sum(L[j,:j]**2))
        
        for i in range(j+1, n):
            L[i,j] = (A[i,j] - np.sum(L[i, :j] * L[j, :j])) / L[j,j]
                         
    return L
                         

A = np.random.randn(5, 5)
A = A @ A.T
L = cholesky(A)
print(metrics.tdist(L - np.tril(L), np.zeros(A.shape)))
print(metrics.tdist(L @ L.T, A))

0.0
1.8444410139024814e-15


# The LDL Decomposition

Let $A \in \mathbb{R}^{n*n}$ a symmetric positive definite matrix.  
$A$ can de decomposed as:
$$A = LDL^T$$
with $L \in \mathbb{R}^{n*n}$ a lower unit triangular matrix and $D \in \mathbb{R}^{n*n}$ a diagonal matrix.  
This is a modified version of the Cholsky decomposition that doesn't need square roots.

$$D_{jj} = A_{jj} - \sum_{k=1}^{j-1} L_{jk}^2D_{kk}$$
$$L_{ij} = \frac{1}{D_{jj}}(A_{ij} - \sum_{k=1}^{j-1}L_{ik}L_{jk}D_{kk}) \text{ for } i > j$$

In [3]:
def cholesky_ldl(A):
    n = len(A)
    L = np.eye(n)
    d = np.zeros(n)
    
    for j in range(n):
        d[j] = A[j,j] - np.sum(L[j,:j]**2 * d[:j])
        
        for i in range(j+1, n):
            L[i,j] = (A[i,j] - np.sum(L[i,:j]*L[j,:j]*d[:j])) / d[j]
                         
    return L, d
                         

A = np.random.randn(5, 5)
A = A @ A.T
L, d = cholesky_ldl(A)
print(metrics.tdist(L - np.tril(L), np.zeros(A.shape)))
print(metrics.tdist(np.diag(L), np.ones(len(L))))
print(metrics.tdist(L @ np.diag(d) @ L.T, A))

0.0
0.0
6.329245045284193e-16


# Solve a linear system

Find $x$ such that:

$$Ax = b$$

Compute the cholesky decomposition of $A$

$$A = LL^T$$
$$LL^Tx = b$$

Solve the lower triangular system:

$$Ly = b$$

Solve the upper triangular system:

$$L^Tx = y$$

In [4]:
def cholesky_system(A, b):
    L = cholesky(A)
    y = solve_triangular(L, b, lower=True)
    x = solve_triangular(L.T, y, lower=False)
    return x

A = np.random.randn(5, 5)
A = A @ A.T
b = np.random.randn(5)
x = cholesky_system(A, b)
print(metrics.tdist(A @ x, b))

9.326416937701413e-14
